In [6]:
%%bash
date

########################################
##### need to specify ################## 
hg19_path=/path/to/save/hg19/fasta/
projPath=/path/to/save/project/
cores=4

histNames=(C_H3K27me3 \
           C_H3K4me3 \
           C_IgG) 
########################################
########################################


##################################################################
## Step 1. fastqc
##################################################################
outPath=${projPath}/processed_data/CUT_RUN/Part1_alignment/

##== FASTQC ==##
mkdir -p ${outPath}/fastqFileQC/
     
# Loop through each histone marker name
for histName in "${histNames[@]}"
do
    fastqc -o ${outPath}/fastqFileQC/ -f fastq ${outPath}/fastq/*${histName}*.fastq.gz > ${outPath}/fastqFileQC/${histName}.log 2>&1
done

##################################################################
## Step 2. trim and redo fastqc
##################################################################
mkdir -p ${outPath}/alignment/trimming

# Loop through each histone marker name
for histName in "${histNames[@]}"
do
    trim_galore --cores ${cores} --fastqc --paired \
    --output_dir ${outPath}/alignment/trimming/ \
    ${outPath}/fastq/*${histName}*_R1*.fastq.gz \
    ${outPath}/fastq/*${histName}*_R2*.fastq.gz &> ${outPath}/alignment/trimming/${histName}_trimming.log
done

##################################################################
## Step 3. bowtie2 align
##################################################################
bowtie2Index=${hg19_path}/bowtie2Index

mkdir -p ${outPath}/alignment/sam/bowtie2_summary
mkdir -p ${outPath}/alignment/bam
mkdir -p ${outPath}/alignment/bed
mkdir -p ${outPath}/alignment/bedgraph

# Loop through each histone marker name
for histName in "${histNames[@]}"
do
    bowtie2 --very-sensitive --local --very-sensitive-local --no-unal --no-mixed --no-discordant --phred33 \
        -I 10 -X 700 -p ${cores} -x ${bowtie2Index}/btIndex \
        -1 ${outPath}/alignment/trimming/*${histName}*_R1*.fq.gz \
        -2 ${outPath}/alignment/trimming/*${histName}*_R2*.fq.gz \
        -S ${outPath}/alignment/sam/${histName}_bowtie2.sam &> ${outPath}/alignment/sam/bowtie2_summary/${histName}_bowtie2.txt
done

date

Tue 16 Jan 2024 06:38:38 PM CST
Tue 16 Jan 2024 07:09:42 PM CST
Tue 16 Jan 2024 07:20:10 PM CST
